- https://github.com/aamini/introtodeeplearning/
- https://github.com/pytorch/examples/tree/master/word_language_model
- http://karpathy.github.io/2015/05/21/rnn-effectiveness/
- https://github.com/karpathy/char-rnn

# LSTM

<img src="https://miro.medium.com/max/4136/1*SKGAqkVVzT6co-sZ29ze-g.png" width="70%"> 

- RNN učimo generirati tekst
- slučaj 1: "_na <span style="color:blue">nebu</span> su <span style="color:blue">oblaci</span>"
- slučaj 2: "_Rodio sam se u <span style="color:blue">Francuskoj</span>. Tamo sam živio do desete godine. Stoga tečno pričam <span style="color:blue">francuski</span>._"
- razmak između bitnih informacija postaje velik

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-shorttermdepdencies.png" width="60%"> 

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png" width="60%"> 


- LSTM (Long Short Term Memory) su varijanta RNN
- imaju mogućnost učiti long-term dependecies

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png" width="60%"> 

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png" width="60%"> 

- horizontalna linija koja prenosi informacije
- vrata (gates) koja propuštaju ili ne informaciju
- vrata su sigmoid + $\times$
- sigmoid funkcija vrati postotak koliko informacije propustiti
1. Prva odluka: koju informaciju propustiti iz prošlog stanja: *forget gate*
    - $f_t = \sigma(W_f [h_{t-1},x_t] + b_f)$
2. Druga odluka: koju novu informaciju uzeti:
    - sigmoid sloj: koje vrijednosti ćemo ažurirati
    - $i_t = \sigma(W_i[h_{t-1},x_t] + b_i)$
    - tanh sloj: kreira novi vektor koji je kandidat za sljedeće stanje
    - $\hat{C}_t = \tanh{(W_c[h_{t-1},x_t]+ b_c)}$
3. Ažuriranje stanja $C_{t-1} \rightarrow C_t$
    - $C_t = f_t\cdot C_{t-1} + i_t\cdot  \hat{C}_t$
4. Odluka o izlazu
    - $o_t = \sigma(W_0[h_{t-1},x_t] + b_o)$
    - $h_t = o_t\cdot \tanh{C_t}$
    
<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png" width="60%">


## Zadatak:

Po uzoru na Karpathyjev model RNN-a koji je naučen generirati stihove s obzirom na Shakespeare-ov korpus, napravite model koji će naučiti generirati stihove s obzirom na Cesarićev korpus. 

In [ ]:
#Prateći tutorial: https://www.youtube.com/watch?v=VAMKuRAh2nc

In [96]:
import tensorflow as tf
import pandas as pd
import string
import requests

In [97]:
data = open('./cesaric/train.txt', 'rb').read().decode(encoding='utf-8')
data = data.split('\r')

In [98]:
for i in range(len(data)):
    print(data[i])

Tiho, o tiho govori mi jesen:

Šuštanjem lišća i šapatom kiše.

Al zima srcu govori još tiše.

I kada sniježi, a spušta se tama,

U pahuljama tišina je sama.



Vedri se nebo. Sunce se rađa.

Plovi iz luke jedna lađa;

Jedna što dugo stajaše u doku,

Sva izbijena, s ranama na boku.

More, ko mati, vuče je na krilo.

Ljulja je, šapće: Ništa nije bilo.



Ne dopire do moga srca pjev

Tih ptica što se javljaju sa grana.

Ja čujem uhom, al ne čujem srcem,

....I lije na uglu petrolejska lampa

Svjetlost crvenkastožutu

Na debelo blato kraj staroga plota

I dvije, tri cigle na putu.



I uvijek ista sirotinja uđe

U njezinu svjetlost iz mraka,

I s licem na kojem su obično brige

Pređe je u par koraka.



A jedne večeri nekoga nema,

A moro bi proć;

I lampa gori,

I gori u magli,

I već je noć.



I nema ga sutra, ni preksutra ne,

I vele da bolestan leži,

I nema ga mjesec, I nema ga dva,

I zima je već,

I sniježi...



A prolaze kao i dosada ljudi

I maj već miriše -

A njega nema, i ne

In [99]:
data = " ".join(data)
data

'Tiho, o tiho govori mi jesen: \nŠuštanjem lišća i šapatom kiše. \nAl zima srcu govori još tiše. \nI kada sniježi, a spušta se tama, \nU pahuljama tišina je sama. \n \nVedri se nebo. Sunce se rađa. \nPlovi iz luke jedna lađa; \nJedna što dugo stajaše u doku, \nSva izbijena, s ranama na boku. \nMore, ko mati, vuče je na krilo. \nLjulja je, šapće: Ništa nije bilo. \n \nNe dopire do moga srca pjev \nTih ptica što se javljaju sa grana. \nJa čujem uhom, al ne čujem srcem, \n....I lije na uglu petrolejska lampa \nSvjetlost crvenkastožutu \nNa debelo blato kraj staroga plota \nI dvije, tri cigle na putu. \n \nI uvijek ista sirotinja uđe \nU njezinu svjetlost iz mraka, \nI s licem na kojem su obično brige \nPređe je u par koraka. \n \nA jedne večeri nekoga nema, \nA moro bi proć; \nI lampa gori, \nI gori u magli, \nI već je noć. \n \nI nema ga sutra, ni preksutra ne, \nI vele da bolestan leži, \nI nema ga mjesec, I nema ga dva, \nI zima je već, \nI sniježi... \n \nA prolaze kao i dosada ljudi 

In [103]:
def CleanText(doc):
    tokens = doc.split()
    table = str.maketrans('','', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens

In [104]:
tokens = CleanText(data)
print(tokens[:50])

['tiho', 'o', 'tiho', 'govori', 'mi', 'jesen', 'šuštanjem', 'lišća', 'i', 'šapatom', 'kiše', 'al', 'zima', 'srcu', 'govori', 'još', 'tiše', 'i', 'kada', 'sniježi', 'a', 'spušta', 'se', 'tama', 'u', 'pahuljama', 'tišina', 'je', 'sama', 'vedri', 'se', 'nebo', 'sunce', 'se', 'rađa', 'plovi', 'iz', 'luke', 'jedna', 'lađa', 'jedna', 'što', 'dugo', 'stajaše', 'u', 'doku', 'sva', 'izbijena', 's', 'ranama']


In [114]:
len(set(tokens))

3064

In [105]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    lines.append(line)
    
print(len(lines))
lines[0]

7576


'tiho o tiho govori mi jesen šuštanjem lišća i šapatom kiše al zima srcu govori još tiše i kada sniježi a spušta se tama u pahuljama tišina je sama vedri se nebo sunce se rađa plovi iz luke jedna lađa jedna što dugo stajaše u doku sva izbijena s ranama na'

In [106]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [107]:
#Every unique world will be assigned an integer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [118]:
tokenizer.word_index

{'i': 1,
 'u': 2,
 'je': 3,
 'se': 4,
 'da': 5,
 'na': 6,
 'a': 7,
 'što': 8,
 'ne': 9,
 'ko': 10,
 'mi': 11,
 'ja': 12,
 'sve': 13,
 'sam': 14,
 'su': 15,
 'od': 16,
 'kao': 17,
 'iz': 18,
 'već': 19,
 'me': 20,
 'ti': 21,
 'sa': 22,
 'o': 23,
 'za': 24,
 'ga': 25,
 'to': 26,
 's': 27,
 'al': 28,
 'kad': 29,
 'će': 30,
 'on': 31,
 'kako': 32,
 'samo': 33,
 'još': 34,
 'život': 35,
 'te': 36,
 'oči': 37,
 'nije': 38,
 'ona': 39,
 'kada': 40,
 'nema': 41,
 'mu': 42,
 'po': 43,
 'srce': 44,
 'tako': 45,
 'tu': 46,
 'ih': 47,
 'one': 48,
 'koje': 49,
 'ni': 50,
 'više': 51,
 'ima': 52,
 'srca': 53,
 'nam': 54,
 'nas': 55,
 'sad': 56,
 'jedna': 57,
 'moje': 58,
 'pred': 59,
 'li': 60,
 'opet': 61,
 'pa': 62,
 'noći': 63,
 'koji': 64,
 'svoje': 65,
 'noć': 66,
 'il': 67,
 'put': 68,
 'vodi': 69,
 'sva': 70,
 'ali': 71,
 'srcu': 72,
 'ljudi': 73,
 'l': 74,
 'nju': 75,
 'kroz': 76,
 'moja': 77,
 'gle': 78,
 'svoj': 79,
 'bez': 80,
 'dana': 81,
 'njem': 82,
 'posve': 83,
 'tiho': 84,
 'ruke': 

In [123]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]

In [124]:
vocab_size = len(tokenizer.word_index) + 1

In [125]:
#one hot-encoding
y = to_categorical(y, num_classes=vocab_size)

In [130]:
seq_length = X.shape[1]

In [132]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [134]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            153250    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 3065)              309565    
Total params: 613,715
Trainable params: 613,715
Non-trainable params: 0
_________________________________________________________________


In [135]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [136]:
model.fit(X,y, batch_size= 256, epochs = 100)

Epoch 1/100
30/30 [==============================] - 8s 168ms/step - loss: 7.7309 - accuracy: 0.0255
Epoch 2/100
30/30 [==============================] - 5s 163ms/step - loss: 7.1245 - accuracy: 0.0458
Epoch 3/100
30/30 [==============================] - 5s 166ms/step - loss: 7.0329 - accuracy: 0.0465
Epoch 4/100
30/30 [==============================] - 5s 162ms/step - loss: 7.0083 - accuracy: 0.0465
Epoch 5/100
30/30 [==============================] - 5s 167ms/step - loss: 7.0011 - accuracy: 0.0465
Epoch 6/100
30/30 [==============================] - 5s 170ms/step - loss: 6.9921 - accuracy: 0.0465
Epoch 7/100
30/30 [==============================] - 5s 163ms/step - loss: 6.9626 - accuracy: 0.0465
Epoch 8/100
30/30 [==============================] - 5s 161ms/step - loss: 6.8504 - accuracy: 0.0465
Epoch 9/100
30/30 [==============================] - 5s 165ms/step - loss: 6.7363 - accuracy: 0.0465
Epoch 10/100
30/30 [==============================] - 5s 161ms/step - loss: 6.6781 - accura

30/30 [==============================] - 5s 165ms/step - loss: 3.0780 - accuracy: 0.2635
Epoch 82/100
30/30 [==============================] - 5s 160ms/step - loss: 3.0194 - accuracy: 0.2713
Epoch 83/100
30/30 [==============================] - 5s 161ms/step - loss: 2.9661 - accuracy: 0.2821
Epoch 84/100
30/30 [==============================] - 5s 163ms/step - loss: 2.8836 - accuracy: 0.3045
Epoch 85/100
30/30 [==============================] - 5s 160ms/step - loss: 2.7977 - accuracy: 0.3270
Epoch 86/100
30/30 [==============================] - 5s 162ms/step - loss: 2.7254 - accuracy: 0.3433
Epoch 87/100
30/30 [==============================] - 5s 166ms/step - loss: 2.6671 - accuracy: 0.3564
Epoch 88/100
30/30 [==============================] - 5s 162ms/step - loss: 2.6063 - accuracy: 0.3677
Epoch 89/100
30/30 [==============================] - 5s 162ms/step - loss: 2.5268 - accuracy: 0.3866
Epoch 90/100
30/30 [==============================] - 5s 158ms/step - loss: 2.4517 - accuracy: 

In [143]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []
    
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')
        
        y_predict = model.predict_classes(encoded)
        
        predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == y_predict:
                predicted_word = word
                break
        seed_text = seed_text + ' ' + predicted_word
        text.append(predicted_word)
    return ' '.join(text)

In [146]:
seed_text = lines[3214]
seed_text

'da pristaje čun što hoćeš čuh iz tame glas ja podigoh glavu i rekoh preko imaš li putnicu upita haron nasmiješih se pokazah svoje patnjama izbrazdano srce on klimnu glavom i prihvati veslo već stupih nogom u njegov čun tad začuh iza leđa krik odakle poznajem taj glas i dvije me'

In [145]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

C:\Users\Petar\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'nikada nada noć i rade i djecu doje i rijetko i nerado i vraća na dvoru diže o ipak nikada se svaka boja praveći me vrijeđa se probudim uglari našli al li jesen naglo pleši pleši to je ne sjedim što je čujem uhom što su ljubav duše lijene što slušah tren a pored ocvalih lijeha nađoh se ispod snuždenih streha on je već dorica je već med dotiče ati sirotinja lebde i stol u ne izvan nađe nomad i pogled kako smiju kako kako već doć u pahuljama sve se lako u riznici more a kad se lako djetinjstva'